# Basic EDA and Feature extraction on Dataset

### import libraries

In [1]:
import os
import sys
import subprocess
import numpy as np
import seaborn as sns
import json
import pandas as pd
import shutil
import librosa.display
import matplotlib
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm

### import data

In [2]:
train_csv = pd.read_csv("/home/abhijeet/Desktop/extracted/train.csv")
train_org = pd.read_csv("/home/abhijeet/Desktop/extracted/train_original.csv")


In [3]:
train_csv.head()

,id,path,file_status,covid_status,date,str_date,status
0,iOx0zPyPiMZjCWXuf3RGu8HKq8k1,/home/abhijeet/Desktop/extracted/20210630/iOx0...,Y,positive_mild,20210630,2021-06-30,positive
1,gFmySpqfQ7YrYqMrmrMPO7AzmBm1,/home/abhijeet/Desktop/extracted/20210630/gFmy...,Y,positive_mild,20210628,2021-06-28,positive
2,padXEGczMRRKmLRVc68rEKI142B3,/home/abhijeet/Desktop/extracted/20210630/padX...,Y,positive_mild,20210628,2021-06-28,positive
3,ymX9t6DWDHZTs28MODDSxDaOj4n2,/home/abhijeet/Desktop/extracted/20210630/ymX9...,Y,positive_mild,20210628,2021-06-28,positive
4,loK2K444HrZUD8Plzr6q3i8y4Xg1,/home/abhijeet/Desktop/extracted/20210630/loK2...,Y,positive_mild,20210628,2021-06-28,positive


In [4]:
train_csv['status'].unique()

array(['positive', 'negative'], dtype=object)

In [5]:
train_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2312 entries, 0 to 2311
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            2312 non-null   object
 1   path          2312 non-null   object
 2   file_status   2312 non-null   object
 3   covid_status  2312 non-null   object
 4   date          2312 non-null   int64 
 5   str_date      2312 non-null   object
 6   status        2312 non-null   object
dtypes: int64(1), object(6)
memory usage: 126.6+ KB


In [6]:
train_csv['path'].loc[1][-44:-16]

'gFmySpqfQ7YrYqMrmrMPO7AzmBm1'

In [7]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1,53 ):
    header += f' mfcc{i}'
for i in range(1,53 ):
    header += f' delta1_mfcc{i}'
for i in range(1,53 ):
    header += f' delta2_mfcc{i}'

header += ' label'
header = header.split()

file = open('/home/abhijeet/Desktop/train_exp.csv', 'w')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
for i in tqdm(range(2312)):
    try :
        filename = train_csv['path'][i]
        y,sr = librosa.load(filename, sr =22050)
        y_log = librosa.power_to_db(y, ref=np.max)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y =y_log, sr =sr, hop_length =1024, n_mfcc=52)
        delta1_mfcc = librosa.feature.delta(mfcc, order=1)
        delta2_mfcc = librosa.feature.delta(mfcc, order=2)
        to_append = f'{filename[:-3].replace(".", "")} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        for f in delta1_mfcc:
            to_append += f' {np.mean(f)}'
        for g in delta2_mfcc:
            to_append += f' {np.mean(g)}'
        
        file = open('/home/abhijeet/Desktop/train_exp.csv', 'a')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
    except :
        continue
# y=[]; chroma_stft =np.ndarray(0);rmse =[]; spec_cent =[];spec_bw =[];
# rollof =[]; zcr =[];mfcc_n = [];sr=[]
# for i in range(0, 2312):
#         filename = train_csv['path'][i]
#         s,sr= librosa.load(filename, mono=True, duration=5)
#         y+=[s]
#         chroma_stft.append(np.array(librosa.feature.chroma_stft(y=y, sr=sr)))
#         rmse += [librosa.feature.rms(y=y)]
#         spec_cent += [librosa.feature.spectral_centroid(y=y, sr=sr)]
#         spec_bw += [librosa.feature.spectral_bandwidth(y=y, sr=sr)]
#         rolloff +=[ librosa.feature.spectral_rolloff(y=y, sr=sr)]
#         zcr += [librosa.feature.zero_crossing_rate(y)]
#         mfcc = librosa.feature.mfcc(y=y, sr=sr)
#         for e in mfcc :
#             mfcc_n += [np.mean(e)]
#         print('..')



 14%|█████▍                                  | 315/2312 [02:06<13:39,  2.44it/s]/home/abhijeet/anaconda3/envs/mini/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: Trying to estimate tuning from empty frequency set.
  return f(*args, **kwargs)
 22%|████████▋                               | 499/2312 [03:38<10:15,  2.95it/s]/home/abhijeet/anaconda3/envs/mini/lib/python3.9/site-packages/librosa/util/decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=1882
  return f(*args, **kwargs)
100%|███████████████████████████████████████| 2312/2312 [16:08<00:00,  2.39it/s]


In [25]:
# data['filename'][0][42:70]


data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc,delta1_mfcc,delta2_mfcc,id
0,/home/abhijeet/Desktop/extracted/20210630/iOx0...,0.571208,0.017260,1816.135002,1825.064974,3654.356836,0.329156,"[[392.25378, 510.57602, 511.05762, 512.76996, ...","[[8.109168, 8.109168, 8.109168, 8.109168, 8.10...","[[-7.390783, -7.390783, -7.390783, -7.390783, ...",iOx0zPyPiMZjCWXuf3RGu8HKq8k1
1,/home/abhijeet/Desktop/extracted/20210630/gFmy...,0.485491,0.055333,1803.227275,1514.007823,3375.087099,0.179753,"[[396.82816, 503.98602, 487.64447, 488.57724, ...","[[6.449207, 6.449207, 6.449207, 6.449207, 6.44...","[[-4.2647905, -4.2647905, -4.2647905, -4.26479...",gFmySpqfQ7YrYqMrmrMPO7AzmBm1
2,/home/abhijeet/Desktop/extracted/20210630/padX...,0.575894,0.031052,2309.826271,1891.974883,4314.941977,0.260138,"[[438.0378, 479.25906, 501.19778, 457.29272, 4...","[[4.3994756, 4.3994756, 4.3994756, 4.3994756, ...","[[-1.022295, -1.022295, -1.022295, -1.022295, ...",padXEGczMRRKmLRVc68rEKI142B3
3,/home/abhijeet/Desktop/extracted/20210630/ymX9...,0.525386,0.044825,2033.952673,1637.164773,3715.567276,0.289644,"[[393.78552, 475.3109, 480.59088, 498.69507, 4...","[[5.560875, 5.560875, 5.560875, 5.560875, 5.56...","[[-6.8439403, -6.8439403, -6.8439403, -6.84394...",ymX9t6DWDHZTs28MODDSxDaOj4n2
4,/home/abhijeet/Desktop/extracted/20210630/loK2...,0.571209,0.041385,2294.375713,1698.955349,4093.764134,0.247430,"[[457.9194, 558.6368, 547.3567, 560.7844, 548....","[[1.6334574, 1.6334574, 1.6334574, 1.6334574, ...","[[-6.542204, -6.542204, -6.542204, -6.542204, ...",loK2K444HrZUD8Plzr6q3i8y4Xg1


In [8]:
data = pd.read_csv("/home/abhijeet/Desktop/train_exp.csv")
data['id']=data.filename.apply(lambda x: x[42:70])
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,delta2_mfcc45,delta2_mfcc46,delta2_mfcc47,delta2_mfcc48,delta2_mfcc49,delta2_mfcc50,delta2_mfcc51,delta2_mfcc52,label,id
0,/home/abhijeet/Desktop/extracted/20210630/iOx0...,0.571208,0.017260,1816.135002,1825.064974,3654.356836,0.329156,473.012207,71.971306,9.502202,...,0.000890,0.010749,-0.003898,-0.008286,-0.000035,0.005898,-0.020526,-0.017116,NaN,iOx0zPyPiMZjCWXuf3RGu8HKq8k1
1,/home/abhijeet/Desktop/extracted/20210630/gFmy...,0.485491,0.055333,1803.227275,1514.007823,3375.087099,0.179753,505.989502,60.505222,-7.180005,...,-0.010931,-0.035267,-0.015948,-0.023493,-0.030032,-0.032335,-0.052866,-0.048199,NaN,gFmySpqfQ7YrYqMrmrMPO7AzmBm1
2,/home/abhijeet/Desktop/extracted/20210630/padX...,0.575894,0.031052,2309.826271,1891.974883,4314.941977,0.260138,452.194244,58.015324,6.083772,...,-0.021170,-0.024662,0.013740,0.014496,-0.011968,-0.038552,-0.050468,-0.041814,NaN,padXEGczMRRKmLRVc68rEKI142B3
3,/home/abhijeet/Desktop/extracted/20210630/ymX9...,0.525386,0.044825,2033.952673,1637.164773,3715.567276,0.289644,431.051880,52.567032,3.823662,...,-0.019847,-0.022493,-0.009439,0.000225,-0.015788,-0.027721,-0.026349,-0.003966,NaN,ymX9t6DWDHZTs28MODDSxDaOj4n2
4,/home/abhijeet/Desktop/extracted/20210630/loK2...,0.571209,0.041385,2294.375713,1698.955349,4093.764134,0.247430,483.063843,54.383022,-1.542054,...,-0.085102,-0.015867,-0.069085,-0.061387,-0.050960,-0.040289,-0.030106,-0.036003,NaN,loK2K444HrZUD8Plzr6q3i8y4Xg1


In [ ]:
# cmap = plt.get_cmap('inferno')
# tot_rows = train_csv.shape[0]
# for i in range(tot_rows):
# #     source = train_csv['file_properties'][i]
#     filename = train_csv['path'][i]
#     y,sr = librosa.load(filename, sr = 22050)
#     plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
#     plt.axis('off');
#     plt.savefig("/home/abhijeet/Desktop/extracted/png/" + filename[-53:-45] + "/"+ filename[-44:-16]+  "-mfcc.png")
#     plt.clf()

In [18]:
tdf = pd.merge(train_csv ,data ,on ='id')
tdf.head()

,id,path,file_status,covid_status,date,str_date,status,filename,chroma_stft,rmse,...,delta2_mfcc44,delta2_mfcc45,delta2_mfcc46,delta2_mfcc47,delta2_mfcc48,delta2_mfcc49,delta2_mfcc50,delta2_mfcc51,delta2_mfcc52,label
0,iOx0zPyPiMZjCWXuf3RGu8HKq8k1,/home/abhijeet/Desktop/extracted/20210630/iOx0...,Y,positive_mild,20210630,2021-06-30,positive,/home/abhijeet/Desktop/extracted/20210630/iOx0...,0.571208,0.017260,...,-0.004861,0.000890,0.010749,-0.003898,-0.008286,-0.000035,0.005898,-0.020526,-0.017116,NaN
1,gFmySpqfQ7YrYqMrmrMPO7AzmBm1,/home/abhijeet/Desktop/extracted/20210630/gFmy...,Y,positive_mild,20210628,2021-06-28,positive,/home/abhijeet/Desktop/extracted/20210630/gFmy...,0.485491,0.055333,...,0.029859,-0.010931,-0.035267,-0.015948,-0.023493,-0.030032,-0.032335,-0.052866,-0.048199,NaN
2,padXEGczMRRKmLRVc68rEKI142B3,/home/abhijeet/Desktop/extracted/20210630/padX...,Y,positive_mild,20210628,2021-06-28,positive,/home/abhijeet/Desktop/extracted/20210630/padX...,0.575894,0.031052,...,-0.015553,-0.021170,-0.024662,0.013740,0.014496,-0.011968,-0.038552,-0.050468,-0.041814,NaN
3,ymX9t6DWDHZTs28MODDSxDaOj4n2,/home/abhijeet/Desktop/extracted/20210630/ymX9...,Y,positive_mild,20210628,2021-06-28,positive,/home/abhijeet/Desktop/extracted/20210630/ymX9...,0.525386,0.044825,...,-0.000107,-0.019847,-0.022493,-0.009439,0.000225,-0.015788,-0.027721,-0.026349,-0.003966,NaN
4,loK2K444HrZUD8Plzr6q3i8y4Xg1,/home/abhijeet/Desktop/extracted/20210630/loK2...,Y,positive_mild,20210628,2021-06-28,positive,/home/abhijeet/Desktop/extracted/20210630/loK2...,0.571209,0.041385,...,-0.057545,-0.085102,-0.015867,-0.069085,-0.061387,-0.050960,-0.040289,-0.030106,-0.036003,NaN


In [19]:
tdf = tdf.drop(['file_status','covid_status','str_date','filename','label'],axis =1)

In [20]:
tdf.to_csv("/home/abhijeet/Desktop/train2_exp.csv",index= False)

In [21]:
df = pd.read_csv("/home/abhijeet/Desktop/train2_exp.csv",sep=',')

In [22]:
df.head()

,id,path,date,status,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,...,delta2_mfcc43,delta2_mfcc44,delta2_mfcc45,delta2_mfcc46,delta2_mfcc47,delta2_mfcc48,delta2_mfcc49,delta2_mfcc50,delta2_mfcc51,delta2_mfcc52
0,iOx0zPyPiMZjCWXuf3RGu8HKq8k1,/home/abhijeet/Desktop/extracted/20210630/iOx0...,20210630,positive,0.571208,0.017260,1816.135002,1825.064974,3654.356836,0.329156,...,0.004345,-0.004861,0.000890,0.010749,-0.003898,-0.008286,-0.000035,0.005898,-0.020526,-0.017116
1,gFmySpqfQ7YrYqMrmrMPO7AzmBm1,/home/abhijeet/Desktop/extracted/20210630/gFmy...,20210628,positive,0.485491,0.055333,1803.227275,1514.007823,3375.087099,0.179753,...,0.037521,0.029859,-0.010931,-0.035267,-0.015948,-0.023493,-0.030032,-0.032335,-0.052866,-0.048199
2,padXEGczMRRKmLRVc68rEKI142B3,/home/abhijeet/Desktop/extracted/20210630/padX...,20210628,positive,0.575894,0.031052,2309.826271,1891.974883,4314.941977,0.260138,...,-0.003277,-0.015553,-0.021170,-0.024662,0.013740,0.014496,-0.011968,-0.038552,-0.050468,-0.041814
3,ymX9t6DWDHZTs28MODDSxDaOj4n2,/home/abhijeet/Desktop/extracted/20210630/ymX9...,20210628,positive,0.525386,0.044825,2033.952673,1637.164773,3715.567276,0.289644,...,0.003627,-0.000107,-0.019847,-0.022493,-0.009439,0.000225,-0.015788,-0.027721,-0.026349,-0.003966
4,loK2K444HrZUD8Plzr6q3i8y4Xg1,/home/abhijeet/Desktop/extracted/20210630/loK2...,20210628,positive,0.571209,0.041385,2294.375713,1698.955349,4093.764134,0.247430,...,-0.048322,-0.057545,-0.085102,-0.015867,-0.069085,-0.061387,-0.050960,-0.040289,-0.030106,-0.036003
